In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import ssl
import json
from requests.adapters import HTTPAdapter
from urllib3.poolmanager import PoolManager
from urllib3.util import ssl_
import time

In [2]:
dep = 44
urlv = "http://annuairesante.ameli.fr/trouver-un-professionnel-de-sante/chirurgiens-dentistes/31-haute-garonne"
base_url = "http://annuairesante.ameli.fr"

In [3]:
response = requests.get(urlv)
soup = BeautifulSoup(response.text,'html.parser')
a_href = soup.findAll('a')

In [4]:
list_url_ville = [] 
list_ville = [] 
a_href = a_href[3:-6]
for link in a_href:
    if link.has_attr('href'):
        list_url_ville.append(link['href'])
        list_ville.append(link.text)

In [5]:
def get_praticien_ville(ville, url):
    response_name = requests.get(base_url + url)
    soup_name = BeautifulSoup(response_name.text, 'html.parser')
    name = soup_name.findAll('a')
    adress = soup_name.findAll(class_="item left adresse")
    
    # contient le nom et le prenom de tous les praticiens de la ville
    name_clean = [] 
    for n in name:
        if n.strong:
            name_clean.append(n)
            
    # liste des informations des praticiens de la ville: nom, adresse et ville
    list_info_praticiens = [] 
    for j in range(len(name_clean)):
        list_info_praticiens.append(name_clean[j].text)
        
    return list_info_praticiens

In [7]:
all_praticiens = [] 
for i in range(len(list_url_ville)):
    praticiens = get_praticien_ville(list_ville[i], list_url_ville[i])
    all_praticiens.extend(praticiens)     

In [ ]:
all_praticiens

In [8]:
dentist=[]

In [ ]:
for i in range(len(all_praticiens)):
        details={}
        url1=f"https://www.doctolib.fr/api/searchbar/autocomplete.json?search={all_praticiens[i]}&output=json"
        response = requests.get(url1)
        res = json.loads(response.text)
        profile = res['profiles']
        details['nom'] = all_praticiens[i]
        if len(profile) == 0:
            details['present_sur_doctolib']="false"
            details['allow_new_patients']= None
            #dentist.append(details) 
        else:
            details['present_sur_doctolib']="true"
            url2=f"https://www.doctolib.fr/profiles/{profile[0]['value']}/faq_details.json"
            response = requests.get(url2)
            patients = json.loads(response.text)
            details['allow_new_patients']=patients['allows_new_patients']
        dentist.append(details)

In [17]:
print(dentist)

[{'nom': 'GALAUP JACQUES', 'present_sur_doctolib': 'false', 'allow_new_patients': None}, {'nom': 'FONTALBAT CHRISTIAN', 'present_sur_doctolib': 'false', 'allow_new_patients': None}, {'nom': 'GAU SOPHIE', 'present_sur_doctolib': 'true', 'allow_new_patients': True}, {'nom': 'SEMPE CONSTANCE', 'present_sur_doctolib': 'false', 'allow_new_patients': None}, {'nom': 'DE BLAY PAULINE', 'present_sur_doctolib': 'false', 'allow_new_patients': None}, {'nom': 'TOCANNE DENIS', 'present_sur_doctolib': 'true', 'allow_new_patients': True}, {'nom': 'BOST ESTELLE', 'present_sur_doctolib': 'false', 'allow_new_patients': None}, {'nom': 'ROUSSET CAROLINE', 'present_sur_doctolib': 'false', 'allow_new_patients': None}, {'nom': 'CRUEGHE BASTIAN', 'present_sur_doctolib': 'false', 'allow_new_patients': None}, {'nom': 'GRAVIE MARIE CHRISTINE', 'present_sur_doctolib': 'false', 'allow_new_patients': None}, {'nom': 'HAFNER BEATRIZ', 'present_sur_doctolib': 'false', 'allow_new_patients': None}, {'nom': 'DECAUX ROMAIN

In [15]:
les_dentistes=pd.DataFrame(dentist)

In [16]:
les_dentistes

,nom,present_sur_doctolib,allow_new_patients
0,GALAUP JACQUES,false,None
1,FONTALBAT CHRISTIAN,false,None
2,GAU SOPHIE,true,True
3,SEMPE CONSTANCE,false,None
4,DE BLAY PAULINE,false,None
5,TOCANNE DENIS,true,True
6,BOST ESTELLE,false,None
7,ROUSSET CAROLINE,false,None
8,CRUEGHE BASTIAN,false,None
9,GRAVIE MARIE CHRISTINE,false,None


In [18]:
les_dentistes.to_csv('dentistes_doctolib.csv', index=False)